In [1]:
import tflearn
import tensorflow as tf
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
logging.debug(1)
logging.info(2)
logging.warning(3)
logging.critical(4)
logging.error(5)

DEBUG:root:1
INFO:root:2
CRITICAL:root:4
ERROR:root:5


In [4]:
import cv2
import os
import numpy as np
from tqdm import tqdm

TRAIN_DIR = "../Cat_Dog/train"

def convert_to_one_hot_label(img_name):
    label = img_name.split(".")[0]
    return [1, 0] if label == "cat" else [0, 1]

def load_train_data(resize_pics=(227, 227)):
    width = resize_pics[0]
    height = resize_pics[1]
    FILE_NAME = "train_data_{}x{}.npy".format(width, height)
    if os.path.exists(FILE_NAME):
        logging.info("load train data")
        train_data = np.load(FILE_NAME)
    else:
        logging.info("create train data")
        train_data = []
        for img_name in tqdm(os.listdir(TRAIN_DIR)):
            one_hot_label = convert_to_one_hot_label(img_name)
            path = os.path.join(TRAIN_DIR, img_name)
            img = cv2.imread(path)
            img = cv2.resize(img, (width, height))
            train_data.append([img, np.array(one_hot_label)])
        logging.info("finish load to memory")
        np.random.shuffle(train_data)
        logging.info("finish shuffle")
        np.save(FILE_NAME, train_data)
        logging.info("finish save to file")
    return train_data

## AlexNet

In [5]:
train_data = load_train_data(resize_pics=(227, 227))

INFO:root:load train data


### Dataset reshaping

In [6]:
X, Y = train_data[:, 0], train_data[:, 1]
X = np.array([i for i in X])
Y = np.array([i for i in Y])
X.shape, Y.shape

((25000, 227, 227, 3), (25000, 2))

In [7]:
MODEL_NAME = "AlexNet_basic"
NUM_CLASS = 2  # Cat, Dog

def alexnet(num_class):
    # Building 'AlexNet'
    network = input_data(shape=[None, 227, 227, 3])
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, num_class, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    
    return network

# Training
model = tflearn.DNN(alexnet(NUM_CLASS), checkpoint_path='model_{}'.format(MODEL_NAME),
                    max_checkpoints=5, tensorboard_verbose=3, tensorboard_dir="log")

In [ ]:
model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
              show_metric=True, batch_size=64, snapshot_step=500,
              snapshot_epoch=False, run_id=MODEL_NAME)

In [9]:
train_data = load_train_data(resize_pics=(128, 128))

INFO:root:create train data
100%|█████████████████████████████████| 25000/25000 [04:52<00:00, 84.22it/s]
INFO:root:finish load to memory
INFO:root:finish shuffle
INFO:root:finish save to file


### Dataset reshaping

In [11]:
train_data = np.array(train_data)

In [12]:
X, Y = train_data[:, 0], train_data[:, 1]
X = np.array([i for i in X])
Y = np.array([i for i in Y])
X.shape, Y.shape

((25000, 128, 128, 3), (25000, 2))

In [21]:
MODEL_NAME = "AlexNet_half"
NUM_CLASS = 2  # Cat, Dog
tf.reset_default_graph()

def alexnet(num_class):
    # Building 'AlexNet'
    network = input_data(shape=[None, 128, 128, 3])
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 1024, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 1024, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, num_class, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    
    return network

# Training
model = tflearn.DNN(alexnet(NUM_CLASS), checkpoint_path='checkpoint/model_{}'.format(MODEL_NAME),
                    max_checkpoints=5, tensorboard_verbose=0, tensorboard_dir="log")

In [22]:
model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
              show_metric=True, batch_size=32, snapshot_step=200,
              snapshot_epoch=False, run_id=MODEL_NAME)
model.save("model/" + MODEL_NAME)

Training Step: 7040  | total loss: 0.72174 | time: 38.292s
| Momentum | epoch: 010 | loss: 0.72174 - acc: 0.4815 -- iter: 22500/22500


In [23]:
MODEL_NAME = "AlexNet_half_2"
NUM_CLASS = 2  # Cat, Dog
tf.reset_default_graph()

def alexnet(num_class):
    # Building 'AlexNet'
    network = input_data(shape=[None, 128, 128, 3])
    network = conv_2d(network, 96, 11, strides=4, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 256, 5, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 384, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 3, strides=2)
    network = local_response_normalization(network)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='tanh')
    network = dropout(network, 0.5)
    network = fully_connected(network, num_class, activation='softmax')
    network = regression(network, optimizer='momentum',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    
    return network

# Training
model = tflearn.DNN(alexnet(NUM_CLASS), checkpoint_path='checkpoint/model_{}'.format(MODEL_NAME),
                    max_checkpoints=5, tensorboard_verbose=0, tensorboard_dir="log")

model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
              show_metric=True, batch_size=64, snapshot_step=200,
              snapshot_epoch=False, run_id=MODEL_NAME)
model.save("model/" + MODEL_NAME)

Training Step: 3520  | total loss: 0.50641 | time: 34.444s
| Momentum | epoch: 010 | loss: 0.50641 - acc: 0.7509 -- iter: 22500/22500


In [24]:
model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
              show_metric=True, batch_size=64, snapshot_step=200,
              snapshot_epoch=False, run_id=MODEL_NAME)
model.save("model/" + MODEL_NAME)

Training Step: 7040  | total loss: 0.26386 | time: 35.391s
| Momentum | epoch: 020 | loss: 0.26386 - acc: 0.8842 -- iter: 22500/22500


## VGG 16

In [5]:
train_data = load_train_data(resize_pics=(227, 227))

INFO:root:load train data


### Dataset reshaping

In [6]:
X, Y = train_data[:, 0], train_data[:, 1]
X = np.array([i for i in X])
Y = np.array([i for i in Y])
X.shape, Y.shape

((25000, 227, 227, 3), (25000, 2))

In [7]:
MODEL_NAME = "VGG16_basic"
NUM_CLASS = 2
def vgg16(num_class):
    # Building 'VGG Network'
    x = tflearn.input_data(shape=[None, 227, 227, 3], name='input')  # 원래는 224x224
    x = tflearn.conv_2d(x, 64, 3, activation='relu', scope='conv1_1')
    x = tflearn.conv_2d(x, 64, 3, activation='relu', scope='conv1_2')
    x = tflearn.max_pool_2d(x, 2, strides=2, name='maxpool1')

    x = tflearn.conv_2d(x, 128, 3, activation='relu', scope='conv2_1')
    x = tflearn.conv_2d(x, 128, 3, activation='relu', scope='conv2_2')
    x = tflearn.max_pool_2d(x, 2, strides=2, name='maxpool2')

    x = tflearn.conv_2d(x, 256, 3, activation='relu', scope='conv3_1')
    x = tflearn.conv_2d(x, 256, 3, activation='relu', scope='conv3_2')
    x = tflearn.conv_2d(x, 256, 3, activation='relu', scope='conv3_3')
    x = tflearn.max_pool_2d(x, 2, strides=2, name='maxpool3')

    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv4_1')
    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv4_2')
    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv4_3')
    x = tflearn.max_pool_2d(x, 2, strides=2, name='maxpool4')

    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv5_1')
    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv5_2')
    x = tflearn.conv_2d(x, 512, 3, activation='relu', scope='conv5_3')
    x = tflearn.max_pool_2d(x, 2, strides=2, name='maxpool5')

    x = tflearn.fully_connected(x, 4096, activation='relu', scope='fc6')
    x = tflearn.dropout(x, 0.5, name='dropout1')

    x = tflearn.fully_connected(x, 4096, activation='relu', scope='fc7')
    x = tflearn.dropout(x, 0.5, name='dropout2')

    x = tflearn.fully_connected(x, num_class, activation='softmax', scope='fc8')
    x = regression(x, optimizer='adam',
                   loss='categorical_crossentropy',
                   learning_rate=0.001)

    return x

# Training
model = tflearn.DNN(vgg16(NUM_CLASS), checkpoint_path='checkpoint/model_{}'.format(MODEL_NAME),
                    max_checkpoints=1, tensorboard_verbose=0, tensorboard_dir="log")

In [10]:
model.fit(X[:10000], Y[:10000], n_epoch=10, validation_set=0.1, shuffle=True,
        show_metric=True, batch_size=32, snapshot_epoch=False,
        snapshot_step=500, run_id=MODEL_NAME)

---------------------------------
Run id: VGG16_basic
Log directory: log/
---------------------------------
Training samples: 9000
Validation samples: 1000
--


ResourceExhaustedError: OOM when allocating tensor with shape[32,64,227,227]
	 [[Node: conv1_1/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_input/X_0_1/_71, conv1_1/W/read)]]
	 [[Node: Accuracy/Mean/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_200_Accuracy/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv1_1/Conv2D', defined at:
  File "c:\python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\python35\lib\site-packages\traitlets\config\applicati

## ResNet

In [ ]:
train_data = load_train_data(resize_pics=(32, 32))

### Dataset reshaping

In [ ]:
X, Y = train_data[:, 0], train_data[:, 1]
X = np.array([i for i in X])
Y = np.array([i for i in Y])
X.shape, Y.shape

In [ ]:
MODEL_NAME = "ResNet_basic"
NUM_CLASS = 2
# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
BLOCK = 5

def resnet(num_class):
    # Building 'Residual Network'
    
    # Real-time data augmentation
    img_aug = tflearn.ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_crop([32, 32], padding=4)

    # Building Residual Network
    net = tflearn.input_data(shape=[None, 32, 32, 3],
                             data_augmentation=img_aug)
    net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
    net = tflearn.residual_block(net, BLOCK, 16)
    net = tflearn.residual_block(net, 1, 32, downsample=True)
    net = tflearn.residual_block(net, BLOCK-1, 32)
    net = tflearn.residual_block(net, 1, 64, downsample=True)
    net = tflearn.residual_block(net, BLOCK-1, 64)
    net = tflearn.batch_normalization(net)
    net = tflearn.activation(net, 'relu')
    net = tflearn.global_avg_pool(net)
    
    # Regression
    net = tflearn.fully_connected(net, num_class, activation='softmax')
    mom = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
    net = tflearn.regression(net, optimizer=mom,
                             loss='categorical_crossentropy')
    
    return net

# Training
model = tflearn.DNN(resnet(NUM_CLASS), checkpoint_path='model_{}'.format(MODEL_NAME),
                    max_checkpoints=5, tensorboard_verbose=3, tensorboard_dir="log")

if False:
    model.fit(X, Y, n_epoch=10, validation_set=0.1,
              snapshot_epoch=False, snapshot_step=500,
              show_metric=True, batch_size=128, shuffle=True,
              run_id=MODEL_NAME)